# Adding Memory to a Chain (Part 2): Creating the Chain

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [ ]:
pip show langchain

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.runnables import chain

from langchain_openai import ChatOpenAI

from langchain.memory import ConversationSummaryMemory

from operator import itemgetter

In [ ]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [ ]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

In [ ]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(),
                                        memory_key="message_log")

In [ ]:
question = "Can you give me an interesting fact I probably didn't know about?"
# question = "Can you elaborate a bit more on this fact?"

In [ ]:
dictionary_output = RunnablePassthrough.assign(
    message_log = RunnableLambda(chat_memory.load_memory_variables) | 
    itemgetter('message_log')).invoke(
    {'question':question})

In [ ]:
prompt_value_output = prompt_template.invoke(dictionary_output)

In [ ]:
ai_message_output = chat.invoke(prompt_value_output)

In [ ]:
response = StrOutputParser().invoke(ai_message_output)

In [ ]:
chat_memory.save_context(inputs = {'input':question}, 
                         outputs = {'output':response})

In [ ]:
chat_memory.load_memory_variables({})

In [ ]:
chain1 = (
    RunnablePassthrough.assign(
        message_log = RunnableLambda(chat_memory.load_memory_variables) | 
        itemgetter('message_log')) 
    | prompt_template 
    | chat 
    | StrOutputParser()
)

question = "Can you elaborate a bit more on this fact?"

response = chain1.invoke({'question':question})

chat_memory.save_context(inputs = {'input':question}, 
                         outputs = {'output':response})

response

In [ ]:
@chain
def memory_chain(question):
    
    chain1 = (
        RunnablePassthrough.assign(
            message_log = RunnableLambda(chat_memory.load_memory_variables) | 
            itemgetter('message_log')) 
        | prompt_template 
        | chat 
        | StrOutputParser()
    )
    
    chain1.get_graph().print_ascii()

    response = chain1.invoke({'question':question})

    chat_memory.save_context(inputs = {'input':question}, 
                             outputs = {'output':response})

    return response

In [ ]:
memory_chain.invoke("Can you elaborate a bit more on this fact?")